In [1]:
import sys
if 'ipykernel_launcher.py' not in sys.argv[0]:
    manual = False
    scenario = sys.argv[1]
else:
    manual = True
    scenario = 'ref_18'
    %matplotlib inline

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np

work_path = r'../../../'
quetzal_path = work_path + r'quetzal_santo_domingo/model/'

plot_path = work_path + r'plot/scenarios/'
import sys
sys.path.insert(0, '../../../quetzal/')

from quetzal.model import stepmodel

SQLalchemy is not installed. No support for SQL output.


In [3]:
from shapely import geometry

In [4]:
sm =  stepmodel.read_zip(quetzal_path + scenario +  '/assigned.zip')

zones: 100%|██████████| 35/35 [00:21<00:00,  1.65it/s]           


In [5]:
sm.volumes.loc[sm.volumes['origin'] == sm.volumes['destination'], ['car', 'nocar']] = 0 

In [6]:
pt = list(sm.mode_nests.reset_index().set_index('root').loc['pt']['route_type'])
print(pt)

['gondola', 'subway', 'minibus', 'bus', 'concho']


In [7]:
from imp import reload
from quetzal.model import summarymodel
reload(summarymodel)

<module 'quetzal.model.summarymodel' from 'c:\\Users\\mgomezdelarosa\\quetzal_quartz\\Santo Domingo\\quetzal_santo_domingo\\notebooks\\transport\\../../../quetzal\\quetzal\\model\\summarymodel.py'>

In [8]:
sm.loaded_links = sm.links.copy()
sm.summary_link_max(inplace=True)
sm.summary_link_sum(inplace=True)
sm.summary_path_sum(inplace=True, dense=True)
sm.summary_path_average(inplace=True, dense=True)
sm.summary_aggregated_path_average(inplace=True)
sm.summary_earning(inplace=True, dense=True)

KeyError: "['boardings'] not in index"

In [12]:
sm.links.columns

Index(['a', 'trip_id', 'link_sequence', 'b', 'route_id', 'direction_id',
       'route_type', 'route_color', 'road_a', 'road_b', 'road_node_list',
       'road_link_list', 'road_length', 'geometry', 'headway', 'speed',
       'length', 'time', 'route_type_group', 'duration', 'cost', 'origin',
       'destination', 'agency_id', 'volume'],
      dtype='object')

In [ ]:
def od_stack(
    self, 
    segments=['car', 'nocar'], 
    costs=['price', 'time', 'in_vehicle_time', 'in_vehicle_length', 'ntransfers']
):
    sm = self
    los = pd.concat([sm.pt_los, sm.car_los])
    los['mode'] = los['route_type']
    los.loc[los['route_type'].isin(pt), 'mode'] = 'pt'

    columns = []

    los = pd.merge(
        los, sm.volumes[['origin', 'destination'] +  segments], 
        on=['origin', 'destination'], suffixes=['_old', '']
    )
    

    for segment in segments:
        los[(segment, 'volume')] = los[(segment, 'probability')] * los[segment]


    for segment in segments:
        columns.append((segment, 'volume'))
        for service in costs:
            column = (segment, service)
            columns.append(column)
            los[column] = los[(segment, 'probability')] * los[service]

    df = los.groupby(['origin', 'destination', 'mode'])[columns].sum()      
    df.columns = pd.MultiIndex.from_tuples(df.columns)

    # add root (weighted mean of segments)

    df[('root', 'volume')] = sum([df[(segment, 'volume')].fillna(0) for segment in segments])

    weighted_sum = sum(
        [
            df[segment][costs].apply(lambda s: s * df[(segment, 'volume')]).fillna(0)
            for segment in segments
        ]      
    )

    for c in costs: 
        df[('root', c)] = (weighted_sum[c] / df[('root', 'volume')]).fillna(0)

    df.columns.names = 'segment', 'sum'
    return df

In [ ]:
def municipio_stack(sm, *args, **kwargs):
    df = od_stack(sm, *args, **kwargs)
    right = sm.zones[['municipio']]
    proto = df.stack('segment')
    index_names = proto.index.names
    left = proto.reset_index()
    merged = pd.merge(left, right, left_on=['origin'], right_index=True
                     ).set_index(index_names).reset_index('mode').reset_index(drop=True)

    wm = lambda x: np.average(x, weights=merged.loc[x.index, 'volume'])
    # Groupby and aggregate with your dictionary:
    return merged.groupby(['municipio', 'mode']).agg(wm).stack()

In [ ]:
sm.pt_los = sm.los.loc[sm.los['route_type'] != 'car']
sm.car_los = sm.los.loc[sm.los['route_type'] == 'car']
sm.stack_municipio = municipio_stack(sm)
sm.stack_municipio.name = 'average'

In [ ]:
sm.stack_municipio.unstack()

in_vehicle_length  in_vehicle_time  ntransfers  \
municipio               mode                                                   
los alcarrizos          pt          8748.069365      2038.302426    0.752506   
                        walk           0.000000         0.000000    0.000000   
santo domingo de guzman pt          3633.673316       858.897093    0.372876   
                        walk           0.000000         0.000000    0.000000   
santo domingo este      pt          6621.167191      1470.483045    0.584817   
                        walk           0.000000         0.000000    0.000000   
santo domingo norte     pt          6958.384305      1403.213219    0.746537   
                        walk           0.000000         0.000000    0.000000   
santo domingo oeste     pt          4393.139505      1072.844697    0.425241   
                        walk           0.000000         0.000000    0.000000   

                                  price         time      volume  
municipio               mode                                      
los alcarrizos          pt    27.012260  4071.331936  130.297001  
                        walk   6.062261   482.063659   17.225997  
santo domingo de guzman pt    22.100579  2218.113322   31.198137  
                        walk   3.590866   684.537179   62.828580  
santo domingo este      pt    25.193066  3115.969018   26.430478  
                        walk   4.781399   821.542744   62.655024  
santo domingo norte     pt    28.636696  3415.578126  154.421946  
                        walk   6.481755   389.189009   21.213592  
santo domingo oeste     pt    22.339267  2518.821435   22.856374  
                        walk   2.485413   856.168129   96.606811

# Buffer

In [ ]:
def buffer_stack(sm, route_types=()):
    sm = sm.copy()
    r = range(1,11)
    
    heavy_nodes = set(sm.links.loc[sm.links['route_type'].isin(route_types)]['a'])
    for b in r:
        buffer = geometry.MultiPoint(list(sm.nodes.loc[heavy_nodes]['geometry'])).buffer(b * 100)

        def in_buffer_ratio(geometry):
            return (geometry.intersection(buffer).area) / geometry.area

        sm.zones['ib' + str(b * 100)] = sm.zones['geometry'].apply(in_buffer_ratio)
    right = sm.zones[['ib' + str(b * 100) for b in r]].apply(lambda c: c*sm.zones['pop'])
    right.columns = [i * 100 for i in list(r)]
    
    tot = sm.zones['pop'].sum()
    s = right.sum() / tot
    s.name = 'ratio'
    s.index.name = 'distance'
    return s

if True:
    sm.stack_buffer = buffer_stack(sm, route_types=['subway', 'tram', 'express_bus', 'gondola'])

In [ ]:
sm.to_excel(quetzal_path + scenario + '/stacks.xlsx')